In [1]:
import os
import traceback
from pathlib import Path
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.document_loaders import PDFMinerLoader
from tqdm import tqdm
import logging
import datetime

# Configuración del logger
logging.basicConfig(
    filename=f"/home/m/Escritorio/RAG/logs/{str(datetime.datetime.today().strftime('%Y-%m-%d'))}_main.log",  # Guardar logs en un archivo
    level=logging.DEBUG,  # Puedes cambiarlo a INFO en producción
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)
logging.info("Inicio\n")


# Paths
PDF_FOLDER = Path("data")
CHROMA_PATH = "db"

# Embedding model (local)
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"


# def load_pdfs(pdf_folder):
#     logging.info(f"📥 Cargando PDFs desde '{pdf_folder}'...")
#     documents = []
#     for filename in os.listdir(pdf_folder):
#         if filename.endswith(".pdf"):
#             file_path = os.path.join(pdf_folder, filename)
#             try:
#                 loader = PyMuPDFLoader(file_path)
#                 docs = loader.load()
#                 documents.extend(docs)
#                 logging.info(f"✔️ {filename} cargado con {len(docs)} páginas.")
#             except Exception as e:
#                 logging.error(f"❌ Error cargando {filename}: {e}")
#                 traceback.print_exc()
#     return documents


def load_pdfs(pdf_folder):
    logging.getLogger("pdfminer").setLevel(logging.WARNING)
    logging.info(f"📥 Cargando PDFs desde '{pdf_folder}'...")
    documents = []
    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            file_path = os.path.join(pdf_folder, filename)
            try:
                loader = PDFMinerLoader(file_path)
                docs = loader.load()
                documents.extend(docs)
                logging.info(f"✔️ {filename} cargado con {len(docs)} páginas usando PDFMinerLoader.")
            except Exception as e:
                logging.warning(f"⚠️ Error con {filename} usando PDFMinerLoader: {e}")
                logging.info("🔁 Intentando cargar con PyMuPDFLoader...")
                traceback.print_exc()
                try:
                    fallback_loader = PyMuPDFLoader(file_path)
                    docs = fallback_loader.load()
                    documents.extend(docs)
                    logging.info(f"✔️ {filename} cargado con PyMuPDFLoader.")
                except Exception as fallback_error:
                    logging.error(f"❌ Fallo también con fallback en {filename}: {fallback_error}")
                    traceback.print_exc()
    return documents



def split_documents(documents, chunk_size=500, chunk_overlap=50):
    try:
        logging.info("✂️ Dividiendo documentos en chunks...")
        splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        docs = splitter.split_documents(documents)
        logging.info(f"✅ Total de chunks: {len(docs)}")
        return docs
    except Exception as e:
        logging.error(f"❌ Error al dividir documentos: {e}")
        traceback.print_exc()
        return []


def embed_documents(chunks):
    try:
        logging.info(f"🔍 Cargando modelo de embeddings: {EMBEDDING_MODEL_NAME}")
        embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
        return embeddings
    except Exception as e:
        logging.error(f"❌ Error cargando modelo de embeddings: {e}")
        traceback.print_exc()
        return None


def create_vector_store(chunks, embeddings, persist_path):
    try:
        logging.info(f"🧠 Creando base vectorial en: {persist_path}")
        vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=persist_path)
        vectordb.persist()
        logging.info("✅ Base vectorial creada y guardada.")
        return vectordb
    except Exception as e:
        logging.error(f"❌ Error creando base vectorial: {e}")
        traceback.print_exc()
        return None


if __name__ == "__main__":
    logging.info("🚀 Iniciando pipeline de ingesta...")

    documents = load_pdfs(PDF_FOLDER)
    if not documents:
        logging.warning("⚠️ No se cargaron documentos. Finalizando.")
        exit()

    chunks = split_documents(documents)
    if not chunks:
        logging.warning("⚠️ No se generaron chunks. Finalizando.")
        exit()

    embeddings = embed_documents(chunks)
    if not embeddings:
        logging.warning("⚠️ No se pudo crear embeddings. Finalizando.")
        exit()

    vectordb = create_vector_store(chunks, embeddings, CHROMA_PATH)
    if not vectordb:
        logging.warning("⚠️ Fallo al crear la base vectorial.")
    else:
        logging.info("🎉 ¡Ingesta completada correctamente!")


Traceback (most recent call last):
  File "/tmp/ipykernel_255252/889351590.py", line 57, in load_pdfs
    docs = loader.load()
  File "/home/m/.local/lib/python3.10/site-packages/langchain_core/document_loaders/base.py", line 32, in load
    return list(self.lazy_load())
  File "/home/m/.local/lib/python3.10/site-packages/langchain_community/document_loaders/pdf.py", line 682, in lazy_load
    yield from self.parser.lazy_parse(blob)
  File "/home/m/.local/lib/python3.10/site-packages/langchain_community/document_loaders/parsers/pdf.py", line 779, in lazy_parse
    visitor_for_all.process_page(page)
  File "/home/m/.local/lib/python3.10/site-packages/pdfminer/pdfinterp.py", line 1215, in process_page
    self.render_contents(page.resources, page.contents, ctm=ctm)
  File "/home/m/.local/lib/python3.10/site-packages/pdfminer/pdfinterp.py", line 1236, in render_contents
    self.execute(list_value(streams))
  File "/home/m/.local/lib/python3.10/site-packages/pdfminer/pdfinterp.py", line 1